In [ ]:
threads=16
import os
os.environ['OMP_NUM_THREADS']=str(threads)
import torch

In [ ]:
import pandas as pd
import urllib
import pickle
import rassp
from rassp import netutil
from rdkit import Chem
import numpy as np

In [ ]:
# XXX: needed by pickle.load of rassp models

from rassp import msutil,model
import sys
sys.modules['msutil'] = msutil
sys.modules['model'] = model

In [ ]:
s = pd.read_parquet('rassp-public/sample_data/smallmols_cfm_pred_public_sample.parquet')

In [ ]:
s.iloc[0].spect

In [ ]:
s.iloc[0].spect_sparse

In [ ]:
s.iloc[0]

In [ ]:
m = Chem.Mol(s.iloc[1].rdmol)
m

In [ ]:
[ a.GetAtomicNum() for a in m.GetAtoms()]

### TODO 

Z toho pouziva RASSP jen spect a rdmol
rdmol je serializovany rdkit.Chem.Mol

to budeme umet z MSP vyrobit

In [ ]:
base='https://people.cs.uchicago.edu/~ericj/rassp/'
files=[ 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.00000740.model',    
'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.meta',
'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.00001000.model', 
'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.meta',   
'subsetnet_best_candidate_nist_posttrain_cluster.cluster-nist-posttrain.35946017.00000480.model',
'subsetnet_best_candidate_nist_posttrain_cluster.cluster-nist-posttrain.35946017.meta']

for f in files:
    urllib.request.urlretrieve(f'{base}/{f}',f)

In [ ]:
# from original run_rassp.py

use_gpu=True
model_dir='.'

MODELS = {
    'FormulaNet': {
        'checkpoint': os.path.join(model_dir, 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.00000740.model'),
        'meta': os.path.join(model_dir, 'formulanet_best_candidate_pcsim_pretrain.nist-fromscratch-3x9x128.35790555.meta'),

        # NOTE(2023-01-24): FN currently has a reduced set of valid mol constraints bc this is what we trained with
        # and there is currently an outstanding bug where changing N_ATOMS here breaks FN predictions in subtle ways.
        'override_constraints': {
            'n_atom': 48,
            # 'n_formula': 4096,
            'n_formula': 32768,
            # 'n_subset': 12288,
            'n_subset': 49152,
        },
    },
    'SubsetNet': {
        'checkpoint': os.path.join(model_dir, 'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.00001000.model'),
        'meta': os.path.join(model_dir, 'subsetnet_best_candidate_nist_fromscratch.nist-fromscratch-test01-old-1x2048.36688199.meta'),

        # NOTE: this set of params works for SN GPU inference on a single RTX 2080 Ti (12GB VRAM)
        # despite SN being trained on <= 48 atoms like FN, it also scales well to mols up to <= 64 atoms (tested)
        'override_constraints': {
            'n_atom': 64,
            'n_formula': 32768,
            'n_subset': 49152,
        },
    },
}

model_name='FormulaNet'
meta_path = MODELS[model_name]['meta']
ckpt_path = MODELS[model_name]['checkpoint']
override_constraints = MODELS[model_name]['override_constraints']
meta = pickle.load(open(meta_path, 'rb'))

feat_config = meta['featurize_config']
for key, val in override_constraints.items():
    if key == 'n_atom':
        feat_config['MAX_N'] = val
    if key == 'n_formula':
        feat_config['explicit_formulae_config']['max_formulae'] = val
    if key == 'n_subset':
        feat_config['vert_subset_samples_n'] = val

if use_gpu and not torch.cuda.is_available():
    use_gpu = False

predictor = netutil.PredModel(
    meta_path,
    ckpt_path,
    USE_CUDA=use_gpu,
    data_parallel=False,
    featurize_config_update=feat_config,
)


In [ ]:
smiles = ['C1=CNN=C1','CC(C)C#N','CCC(=C)C','CC[N+](=O)[O-]','C1=CSC(=N1)N=C(C(C(F)(F)F)(OC(C(C(F)(F)F)(OC(C(C(F)(F)F)(F)F)(F)F)F)(F)F)F)N'] 

valid_mols = [ Chem.AddHs(Chem.MolFromSmiles(s)) for s in smiles ]

In [ ]:
predictions = predictor.pred(
    valid_mols,
    progress_bar=True,
    normalize_pred=True,
    output_hist_bins=True,
    batch_size=2, # XXX
    dataloader_config={
        'pin_memory': False,
        'num_workers': 0, # XXX
        'persistent_workers': False,
    },
    benchmark_dataloader=False,
)


In [ ]:
shit_pred17 = predictions['pred_binned']

In [ ]:
nist_path = '20210925_NIST_EI_MS_cleaned.msp'

from matchms.importing import load_from_msp
nist = list(load_from_msp(nist_path, metadata_harmonization=False))

In [ ]:
len(nist)

In [ ]:
nist[42].metadata

In [ ]:
Chem.MolToSmiles(Chem.inchi.MolFromInchi(nist[42].metadata['inchi']))

In [ ]:
nist[42].peaks.mz

In [ ]:
nist[42].peaks.intensities

In [ ]:
mol = Chem.AddHs(Chem.MolFromSmiles(nist[42].metadata['smiles']))
len(mol.GetPropNames())

In [ ]:
b = mol.ToBinary()
b

In [ ]:
m2 = Chem.Mol(b)
m2

In [ ]:
import numpy as np
a=np.stack([nist[42].peaks.mz,nist[42].peaks.intensities],axis=1)
a

In [ ]:
bb = np.empty(shape=(a.shape[0],),dtype=object)
bb[:] = list(a)
bb

In [ ]:
np.array(list(a),dtype=object)

In [ ]:
n=nist[42]
np.array([ np.array(p,dtype=object) for p in zip(n.peaks.mz, n.peaks.intensities)])
#[ np.array(p) for p in zip(n.peaks.mz, n.peaks.intensities)]

In [ ]:
# valid_atoms = {1, 6, 7, 8, 9, 15, 16, 17}
# FIXME: rassp/msutil/shared.h
# not acceptable, narrows down the model too much, we need Br, Si at least

# fixed:
valid_atoms = {1, 5, 6, 7, 8, 9, 14, 15, 16, 17, 35, 53}

# requires isotopic data in rassp/msutil/floatmasseval_per_elt_peaks.h

# dtto MAX_N == 48

def mkdf(nist2):
    missing = dict()
    mols = [ ]
    spects = []
    smiles = []
    for i,n in enumerate(nist2):
        if n.metadata.get('smiles') is None:
            print(i,'no smiles')
            continue
        mol = Chem.MolFromSmiles(n.metadata['smiles'])
        if mol is None: 
            print(i,n.metadata['smiles'][:100],'... cant parse')
            continue 
        mol = Chem.AddHs(mol)
        if mol is None: 
            print(i,n.metadata['smiles'],'... cant add Hs')
            continue 
            
        if len(mol.GetAtoms()) > 48:
            print(i,n.metadata['smiles'],'... too many atoms')
            continue
        
        atoms = { a.GetAtomicNum() for a in mol.GetAtoms() }
        if not atoms < valid_atoms:
            for e in atoms - valid_atoms:
                missing[e] = missing.get(e,0)+1
                
            print(i,n.metadata['smiles'],atoms,'... invalid atom')
            continue
        
        mols.append(mol.ToBinary())
        a = np.stack([n.peaks.mz, n.peaks.intensities], axis=1)
        b = np.empty(shape=(a.shape[0],),dtype=object)
        b[:] = list(a)
        spects.append(b)
        smiles.append(n.metadata['smiles'])

    df = pd.DataFrame.from_dict({ 'rdmol' : mols, 'spect' : spects, 'smiles' : smiles })
    return df,missing

In [ ]:
%%capture
trd,missing = mkdf(nist)

In [ ]:
sorted(missing.items(),key=lambda x: x[1])

In [ ]:
test_mask = np.random.choice([True, False], p=[.2,.8], size=len(trd))

In [ ]:
trd[test_mask == False].to_parquet('train.pq')

In [ ]:
td = pd.read_parquet('train.pq')                    

In [ ]:
td.iloc[0].spect

In [ ]:
%%capture
tstd = mkdf(nist[1:1000:2])

In [ ]:
#tstd.to_parquet('test.pq')
trd[test_mask].to_parquet('test.pq')

In [ ]:
tstd

In [ ]:
sys.path.append('./rassp-public/rassp')
from forward_train import train

In [ ]:
import yaml
with open('rassp-public/rassp/expconfig/demo.yaml') as cf:
    exp_config = yaml.load(cf,Loader=yaml.FullLoader)

In [ ]:
exp_config['cluster_config']['data_dir'] = '.'
exp_config['DATALOADER_NUM_WORKERS'] = 8
exp_config['max_epochs'] = 200
exp_config['epoch_size'] = 4096
exp_config['batch_size'] = 4
exp_config['automatic_mixed_precision'] = False # broken
exp_config['featurize_config']['explicit_formulae_config']['max_formulae'] = 27000 # XXX
exp_config['exp_data']['data'][0]['db_filename'] = 'train.pq'
exp_config['exp_data']['data'][1]['db_filename'] = 'test.pq'

with open('exp.yml','w') as y:
    yaml.dump(exp_config,y)

In [ ]:
train('test',exp_config,exp_config_filename='exp.yml',USE_CUDA=True)

In [ ]:
trd[trd.smiles.str.contains('C(Cl)Br',regex=False)]
#trd[trd.smiles.str.contains('Br',regex=False)]

In [ ]:
m=Chem.Mol(trd.iloc[7208].rdmol)
m

In [ ]:
not {a.GetAtomicNum() for a in m.GetAtoms()} < {1,2,6,7,8,9,15,16,17}

In [ ]:
trd = pd.read_parquet('train.pq')

In [ ]:
shit_nist=trd[trd.smiles.str.contains({*smiles},regex=False)]

In [ ]:
idx=4

In [ ]:
shit_nist=trd[trd.smiles == smiles[idx]]
shit_nist

In [ ]:
idx2=0

In [ ]:
import matchms
mzi=np.array(list(shit_nist.iloc[idx2].spect))
shit1 = matchms.Spectrum(mz=mzi[:,0],intensities=mzi[:,1])
mzi=shit_pred17[idx].astype(float)
shit2 = matchms.Spectrum(mz=mzi[:,0],intensities=mzi[:,1])

In [ ]:
from matchms.plotting import plot_spectra_mirror
plot_spectra_mirror(shit1,shit2)

In [ ]:
nist[42]